In [3]:
pip install azure-ai-formrecognizer --pre

     |████████████████████████████████| 149 kB 20.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

In [2]:
endpoint = "<https://formrecognizeride-1.cognitiveservices.azure.com/>"
key = "<6d57b0063a764cc4b71111bbf8cfe6c5>"

In [3]:
form_recognizer_client = FormRecognizerClient(endpoint, AzureKeyCredential(key))

In [4]:
def print_result(invoices, blob_name):
    for idx, invoice in enumerate(invoices):
        print("--------Recognizing invoice {}--------".format(blob_name))
        vendor_name = invoice.fields.get("VendorName")
        if vendor_name:
            print("Vendor Name: {} has confidence: {}".format(vendor_name.value, vendor_name.confidence))
        vendor_address = invoice.fields.get("VendorAddress")
        if vendor_address:
            print("Vendor Address: {} has confidence: {}".format(vendor_address.value, vendor_address.confidence))
        customer_name = invoice.fields.get("CustomerName")
        if customer_name:
            print("Customer Name: {} has confidence: {}".format(customer_name.value, customer_name.confidence))
        customer_address = invoice.fields.get("CustomerAddress")
        if customer_address:
            print("Customer Address: {} has confidence: {}".format(customer_address.value, customer_address.confidence))
        customer_address_recipient = invoice.fields.get("CustomerAddressRecipient")
        if customer_address_recipient:
            print("Customer Address Recipient: {} has confidence: {}".format(customer_address_recipient.value, customer_address_recipient.confidence))
        invoice_id = invoice.fields.get("InvoiceId")
        if invoice_id:
            print("Invoice Id: {} has confidence: {}".format(invoice_id.value, invoice_id.confidence))
        invoice_date = invoice.fields.get("InvoiceDate")
        if invoice_date:
            print("Invoice Date: {} has confidence: {}".format(invoice_date.value, invoice_date.confidence))
        invoice_total = invoice.fields.get("InvoiceTotal")
        if invoice_total:
            print("Invoice Total: {} has confidence: {}".format(invoice_total.value, invoice_total.confidence))
        due_date = invoice.fields.get("DueDate")
        if due_date:
            print("Due Date: {} has confidence: {}".format(due_date.value, due_date.confidence))

In [5]:
# Create the BlobServiceClient object which will be used to get the container_client
connect_str = "<DefaultEndpointsProtocol=https;AccountName=wwstorageide1;AccountKey=Cb3aKpMaW43EU7vQBCmxLQkPEgH4Zlewpm6iIZ9d6YJjrGEpCQtaTYbSp1yRhdfrbSSpETtdLu7uZfAodkzdVw==;EndpointSuffix=core.windows.net>"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Container client for raw container.
raw_container_client = blob_service_client.get_container_client("wwstorageideblob1")

# Container client for processed container
processed_container_client = blob_service_client.get_container_client("processed")

# Get base url for container.
invoiceUrlBase = raw_container_client.primary_endpoint
print(invoiceUrlBase)

https://wwstorageide1.blob.core.windows.net>/wwstorageideblob1


In [6]:
print("\nProcessing blobs...")

blob_list = raw_container_client.list_blobs()
for blob in blob_list:
    invoiceUrl = f'{https://wwstorageide1.blob.core.windows.net/wwstorageideblob1/ACORD-s5-g21702kg21i005-PDF.pdf}/{wwstorageideblob1}'
    print(invoiceUrl)
    poller = form_recognizer_client.begin_recognize_invoices_from_url(invoiceUrl)

    # Get results
    invoices = poller.result()

    # Print results
    print_result(invoices, blob.name)

    # Copy blob to processed
    processed_container_client.upload_blob(blob, blob.blob_type, overwrite=True)

    # Delete blob from raw now that its processed
    raw_container_client.delete_blob(blob)


Processing blobs...


ServiceRequestError: <urllib3.connection.HTTPSConnection object at 0x7f066a86a080>: Failed to establish a new connection: [Errno -2] Name or service not known